In [1]:
import os
import time
import pprint
import argparse
import torch
import numpy as np
import pickle
import utils
import csv

from model.hidden import Hidden
from noise_layers.noiser import Noiser
from average_meter import AverageMeter
from noise_argparser import NoiseArgParser

import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

/home/master/10/jimmyliu/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
m, n = 100, 2
# generate random data, with arbitrary covariance
X = np.random.multivariate_normal([0 for i in range(n)], np.array([[1, 0.8], [0.8, 1]]), size=m)
X = X - X.mean(axis=0) # cener the data

In [13]:
print(X.shape)
C= np.cov(X.T)
print(C.shape)
eig_vals, eig_vecs = np.linalg.eig(C)
print(eig_vals)
D = np.diag(eig_vals) # eig_vals is a vector, but we want a matrix
P = eig_vecs
print(C)
print(P@D@P.T)

torch.Size([100, 2])
(2, 2)
[2.37811949 0.16247392]
[[1.28924364 1.10766075]
 [1.10766075 1.25134977]]
[[1.28924364 1.10766075]
 [1.10766075 1.25134977]]


In [18]:
print(X.shape)
X=torch.tensor(X)
C= torch.cov(X.T)
print(C.shape)
eig_vals, eig_vecs = torch.linalg.eigh(C)
print(eig_vals)
D = torch.diag(eig_vals) # eig_vals is a vector, but we want a matrix
P = eig_vecs
print(C)
print(P@D@P.T)

torch.Size([100, 2])
torch.Size([2, 2])
tensor([0.1625, 2.3781], dtype=torch.float64)
tensor([[1.2892, 1.1077],
        [1.1077, 1.2513]], dtype=torch.float64)
tensor([[1.2892, 1.1077],
        [1.1077, 1.2513]], dtype=torch.float64)


/tmp/ipykernel_32503/228613615.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X=torch.tensor(X)


In [13]:
data_dir='/tmp2/jimmyliu/HiDDeN_experiment/dataset_10000'
current_run='/tmp2/jimmyliu/HiDDeN_experiment/ss_HiDDeN/runs/combined-noise-256-adam 2023.05.20--02-44-07'
batch_size=2
round=True

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
options_file = os.path.join(current_run, 'options-and-config.pickle')
train_options, hidden_config, _ = utils.load_options(options_file)
train_options.train_folder = os.path.join(data_dir, 'val')
train_options.validation_folder = os.path.join(data_dir, 'train')
train_options.batch_size = batch_size
noise_config=[]

In [14]:
checkpoint, chpt_file_name = utils.load_last_checkpoint(os.path.join(current_run, 'checkpoints'))
print(f'Loaded checkpoint from file {chpt_file_name}')

noiser = Noiser(noise_config,device)
model = Hidden(hidden_config, device, noiser, tb_logger=None,train_options=train_options)
utils.model_from_checkpoint(model, checkpoint)
model.encoder_decoder.eval()

Loaded checkpoint from file /tmp2/jimmyliu/HiDDeN_experiment/ss_HiDDeN/runs/combined-noise-256-adam 2023.05.20--02-44-07/checkpoints/combined-noise-256-adam--epoch-33.pyt


EncoderDecoder(
  (encoder): Encoder(
    (conv_layers): Sequential(
      (0): ConvBNRelu(
        (layers): Sequential(
          (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (1): ConvBNRelu(
        (layers): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (2): ConvBNRelu(
        (layers): Sequential(
          (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
        )
      )
      (3): ConvBNRelu(
        (layers): Sequential(
          (0): Conv2d(64, 64, kernel

In [15]:
_, val_data = utils.get_data_loaders(hidden_config, train_options)
file_count = len(val_data.dataset)
if file_count % train_options.batch_size == 0:
            steps_in_epoch = file_count // train_options.batch_size
else:
    steps_in_epoch = file_count // train_options.batch_size + 1

In [16]:
print(file_count)

10000


In [17]:

step = 0
losses_accu={}

ori_list_arr=[]
with torch.no_grad():
    for image, _ in val_data:
        
        image = image.to(device)
        #message = torch.Tensor(np.random.choice([0, 1], (image.shape[0], hidden_config.message_length))).to(device)
        decoded_messages= model.encoder_decoder.decoder(image)
        if step==0:
            ori_list_arr=decoded_messages.detach().cpu()
        else:
            ori_list_arr=torch.cat( (ori_list_arr,decoded_messages.detach().cpu()),0 )



        step += 1
        '''
        if step >=2:
            break
        '''

In [ ]:

if round:
    ruond_ori_list_arr=ori_list_arr.round().clip(0, 1)

miu=torch.mean(ori_list_arr, 0)
round_miu=torch.mean(ruond_ori_list_arr, 0)

In [ ]:
print(miu.mean())
print(round_miu.mean())

In [ ]:
ori_list_arr-=miu

In [ ]:
conv_matrix=torch.cov(ori_list_arr.T)

#plt.matshow(conv_matrix, cmap=plt.cm.Blues)
plt.imshow(conv_matrix,vmin=-0.05,vmax=0.3)
plt.colorbar()

plt.show()



In [ ]:
round_conv_matrix=torch.cov(ruond_ori_list_arr.T)

#plt.matshow(conv_matrix, cmap=plt.cm.Blues)
plt.imshow(round_conv_matrix,vmin=-0.05,vmax=0.3)
plt.colorbar()

plt.show()

In [ ]:
print(round_conv_matrix.T == round_conv_matrix)

In [ ]:
conv_matrix=(conv_matrix.T+conv_matrix)/2
print(torch.all(conv_matrix.T == conv_matrix))

In [ ]:
L, V = torch.linalg.eig(conv_matrix)
#L, V = torch.linalg.eigh(conv_matrix)
L_matrix=torch.diag_embed(1/torch.sqrt(L))
#b=-(L_matrix@V.T).to(torch.float)@(miu-0.5)
b=-(L_matrix@V.T).to(torch.float)@miu
w=(L_matrix@V.T).to(torch.float)

eig_whitening_layer=nn.Linear(48,48)
eig_whitening_layer.weight=nn.Parameter(w)
eig_whitening_layer.bias=nn.Parameter(b)
eig_whitening_layer=eig_whitening_layer.to(device)
eig_whitening_layer.eval()

In [ ]:
#L, V = torch.linalg.eig(conv_matrix)
L, V = torch.linalg.eigh(conv_matrix)
L_matrix=torch.diag_embed(1/torch.sqrt(L))

#b=-(L_matrix@V.T).to(torch.float)@(miu-0.5)
b=-(L_matrix@V.T).to(torch.float)@miu
w=(L_matrix@V.T).to(torch.float)

eigh_whitening_layer=nn.Linear(48,48)
eigh_whitening_layer.weight=nn.Parameter(w)
eigh_whitening_layer.bias=nn.Parameter(b)
eigh_whitening_layer=eigh_whitening_layer.to(device)
eigh_whitening_layer.eval()

In [ ]:

step = 0
losses_accu={}

eig_list_arr=[]

eigh_list_arr=[]
with torch.no_grad():
    for image, _ in val_data:
        
        image = image.to(device)
        #message = torch.Tensor(np.random.choice([0, 1], (image.shape[0], hidden_config.message_length))).to(device)
        decoded_messages= model.encoder_decoder.decoder(image)
        eig_decoded_messages=eig_whitening_layer(decoded_messages)
        eigh_decoded_messages=eigh_whitening_layer(decoded_messages)


        if step==0:
            eig_list_arr=eig_decoded_messages.detach().cpu()
            eigh_list_arr=eigh_decoded_messages.detach().cpu()
        else:
            eig_list_arr=torch.cat( (eig_list_arr,eig_decoded_messages.detach().cpu()),0 )
            eigh_list_arr=torch.cat( (eigh_list_arr,eigh_decoded_messages.detach().cpu()),0 )



        step += 1
       

In [ ]:
print(decoded_messages)

In [ ]:
print(eig_decoded_messages)

In [ ]:
noround_eig_miu=torch.mean(eig_list_arr, 0)
noround_eigh_miu=torch.mean(eigh_list_arr, 0)
'''
if round:
    eig_list_arr=eig_list_arr.round().clip(0, 1)
    eigh_list_arr=eigh_list_arr.round().clip(0, 1)
'''
eig_conv_matrix=torch.cov(eig_list_arr.T)
plt.imshow(eig_conv_matrix,vmin=-0.05,vmax=0.3)
plt.colorbar()

plt.show()

In [ ]:
eigh_conv_matrix=torch.cov(eigh_list_arr.T)
plt.imshow(eigh_conv_matrix,vmin=-0.05,vmax=0.3)
plt.colorbar()

plt.show()

In [ ]:
print(noround_eig_miu.mean())
print(noround_eigh_miu.mean())